In [1]:
%load_ext autotime

import os
import shutil
import tarfile
import numpy as np
import pandas as pd

import scarf
scarf.__version__

'0.23.0'

time: 1.47 s (started: 2022-09-13 23:35:32 +02:00)


In [2]:
scarf.utils.set_verbosity('DEBUG')

time: 15 ms (started: 2022-09-13 23:35:34 +02:00)


In [3]:
def load_datastore():
    fn = os.path.join('scarf_datasets/1K_pbmc_citeseq.zarr.tar.gz')
    out_fn = fn.replace('.tar.gz', '')
    if os.path.isdir(out_fn):
        shutil.rmtree(out_fn)
    tar = tarfile.open(fn, "r:gz")
    tar.extractall(out_fn)
    return scarf.DataStore(out_fn, default_assay='RNA', assay_types={'assay2': 'ADT'} )

ds = load_datastore()

DEBUG: Setting assay RNA to assay type: RNAassay


(RNA) Computing nCells and dropOuts:   0%|                                                                    …

DEBUG: Setting assay assay2 to assay type: ADTassay


(assay2) Computing nCells and dropOuts:   0%|                                                                 …

(RNA) Computing nCounts:   0%|                                                                                …

...

(RNA) Computing RNA_percentMito:   0%|                                                                        …

(RNA) Computing RNA_percentRibo:   0%|                                                                        …

(assay2) Computing nCounts:   0%|                                                                             …

(assay2) Computing nFeatures:   0%|                                                                           …

time: 1.88 s (started: 2022-09-13 23:35:34 +02:00)


In [4]:
ds.auto_filter_cells(show_qc_plots=False)
ds.mark_hvgs(top_n=100, show_plot=False)
ds.run_cell_cycle_scoring()
ds.make_graph(feat_key='hvgs')
ds.run_leiden_clustering()
ds.run_clustering(n_clusters=10)
ds.run_clustering(balanced_cut=True, max_size=100, min_size=10,
                  label='balanced_clusters')
ds.run_umap(n_epochs=100)
ds.run_marker_search(group_key='RNA_cluster')
ds.run_mapping(target_assay=ds.RNA, target_name='selfmap',
                      target_feat_key='hvgs_self', save_k=3)
ds.run_unified_umap(target_names=['selfmap'], n_epochs=100,)
ds.run_topacedo_sampler(cluster_key='RNA_cluster')

ds.run_pseudotime_scoring(source_sink_key="RNA_leiden_cluster", sources=[6], sinks=[3])
ds.run_pseudotime_marker_search(pseudotime_key='RNA_pseudotime')
ds.run_pseudotime_aggregation(
    pseudotime_key='RNA_pseudotime', cluster_label='pseudotime_clusters',
    n_clusters = 15, window_size=50, chunk_size=10)
ds.add_grouped_assay(group_key='pseudotime_clusters', assay_label='PTIME_MODULES')

INFO: 37 cells flagged for filtering out using attribute RNA_nCounts
INFO: 21 cells flagged for filtering out using attribute RNA_nFeatures
INFO: 43 cells flagged for filtering out using attribute RNA_percentMito
INFO: 4 cells flagged for filtering out using attribute RNA_percentRibo
INFO: Setting `min_cells` to 8. Only those genes that are present in atleast this number of cells will be considered HVGs.


(RNA) Computing nCells:   0%|                                                                                 …

(RNA) Computing normed_tot:   0%|                                                                             …

(RNA) Computing sigmas:   0%|                                                                                 …

INFO: 100 genes marked as HVGs
DEBUG: Location (summary_stats_I) already mounted


...

DEBUG: Setting assay PTIME_MODULES to assay type: Assay


(PTIME_MODULES) Computing nCells and dropOuts:   0%|                                                          …

(PTIME_MODULES) Computing nCounts:   0%|                                                                      …

(PTIME_MODULES) Computing nFeatures:   0%|                                                                    …

time: 43.2 s (started: 2022-09-13 23:35:36 +02:00)


In [5]:
indices = ds.z.RNA.normed__I__hvgs.reduction__pca__11__I.ann__l2__50__50__48__4466.knn__11.indices[:]
np.save('knn_indices.npy', indices)

distances = ds.z.RNA.normed__I__hvgs.reduction__pca__11__I.ann__l2__50__50__48__4466.knn__11.distances[:]
np.save('knn_distances.npy', distances)

weights = ds.z.RNA.normed__I__hvgs.reduction__pca__11__I.ann__l2__50__50__48__4466.knn__11['graph__1.0__1.5']['weights'][:]
np.save('knn_weights.npy', weights)

cell_attrs = ds.cells.to_pandas_dataframe(ds.cells.columns, key='I')
cell_attrs['target_classes'] = ds.get_target_classes(target_name='selfmap', reference_class_group='RNA_cluster').values
cell_attrs['mapping_scores'] = next(ds.get_mapping_score(target_name='selfmap'))[1]
cell_attrs.to_csv('cell_attributes.csv')

ds.get_markers(group_key='RNA_cluster', group_id=1).to_csv('markers_cluster1.csv')
ds.export_markers_to_csv(group_key='RNA_cluster', csv_filename='markers_all_clusters.csv')

ds.RNA.feats.to_pandas_dataframe(['names', 'I__RNA_pseudotime__r'], key='I').to_csv('pseudotime_markers_r_values.csv')
np.save('aggregated_feat_idx.npy', ds.z.RNA.aggregated_I_I_RNA_pseudotime.feature_indices[:])
np.save('aggregated_df_top_10.npy', ds.z.RNA.aggregated_I_I_RNA_pseudotime.data[:10])
np.save('pseudotime_clusters.npy', ds.RNA.feats.fetch_all('pseudotime_clusters'))
np.save('ptime_modules_group_1.npy', ds.get_cell_vals(from_assay='PTIME_MODULES', cell_key='I', k='group_1'))

time: 3.83 s (started: 2022-09-13 23:36:19 +02:00)


In [6]:
precalc_markers = pd.read_csv("pseudotime_markers_r_values.csv", index_col=0)
pseudotime_markers = ds.RNA.feats.to_pandas_dataframe(['names', 'I__RNA_pseudotime__r'], key='I')
assert np.alltrue(precalc_markers.index == pseudotime_markers.index)
assert np.alltrue(precalc_markers.names.values == pseudotime_markers.names.values)
assert np.allclose(precalc_markers.I__RNA_pseudotime__r.values, pseudotime_markers.I__RNA_pseudotime__r.values)

time: 31 ms (started: 2022-09-13 23:36:23 +02:00)


In [7]:
ds.RNA.feats.to_pandas_dataframe(['names', 'I__RNA_pseudotime__r'], key='I')

,names,I__RNA_pseudotime__r
6,AL627309.5,0.123055
14,LINC01409,0.026976
16,LINC01128,-0.064882
24,NOC2L,-0.005567
29,HES4,0.066715
...,...,...
36570,MT-ND6,-0.187929
36571,MT-CYB,-0.089457
36580,AL592183.1,0.019249
36581,AC240274.1,-0.000715


time: 31 ms (started: 2022-09-13 23:36:23 +02:00)


In [8]:
# cell_attrs = pd.read_csv('cell_attributes.csv', index_col=0)
# diff = ds.cells.fetch('RNA_pseudotime') - cell_attrs["RNA_pseudotime"].values
# np.all(diff < 1e-3)

time: 0 ns (started: 2022-09-13 23:36:23 +02:00)


In [9]:
# scarf.writers.to_h5ad(ds.RNA, '1K_pbmc_citeseq.h5ad')
# scarf.writers.to_mtx(ds.RNA, '1K_pbmc_citeseq_dir', compress=True)
# writer = scarf.writers.SubsetZarr(in_zarr='1K_pbmc_citeseq.zarr/', out_zarr='test_subset.zarr', cell_idx=[1, 10, 100, 500])

time: 0 ns (started: 2022-09-13 23:36:23 +02:00)
